<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD001_Model_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
# Enable HalvingSearch
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV
# from sklearn.model_selection import GridSearchCV

In [22]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures

In [4]:
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import cross_val_score

# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import SGDRegressor

# from sklearn.pipeline import Pipeline
# from sklearn.compose import TransformedTargetRegressor
# from sklearn.exceptions import ConvergenceWarning

# from sklearn.preprocessing import FunctionTransformer
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.preprocessing import StandardScaler

In [5]:
# from warnings import simplefilter

In [6]:
sns.set_palette('colorblind')

# Data Preparation

In [7]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [8]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [9]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [10]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [11]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [12]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [13]:
X_test = test_last

### Remaining Useful Life (RUL)

In [14]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [15]:
X_train, y_train = proc.X_y_train_divide(train)

In [16]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [17]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [18]:
eval = Evaluation()

In [19]:
search = HyperparameterSearch()

# DummyRegressor

Just to have a baseline on models. The trained models must have a better performance than a dummy.

## Linear RUL

In [24]:
model = Pipeline([
    ('dummy_reg'  ,   DummyRegressor())
])

In [25]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.000,RMSE=-68.879
(CV) R2=-0.029,RMSE=-68.783


In [26]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.604,RMSE=-52.625


## Non-linear RUL

In [32]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':90})))
    ])

In [33]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.000,RMSE=-28.347
(CV) R2=-0.004,RMSE=-28.360


In [34]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.017,RMSE=-41.901


## PolyFeatures + Linear RUL

In [35]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DummyRegressor())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DummyRegressor())])

In [36]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.000,RMSE=-68.879
(CV) R2=-0.029,RMSE=-68.783


In [37]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.604,RMSE=-52.625


## PolyFeatures + Non-Linear RUL

In [38]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':90})))
])

In [39]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.000,RMSE=-28.347
(CV) R2=-0.004,RMSE=-28.360


In [40]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.017,RMSE=-41.901


# KNeighborsRegressor

## Linear RUL

In [42]:
model = Pipeline([
    ('neigh_reg'  ,   KNeighborsRegressor())
])

In [46]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "neigh_reg__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "neigh_reg__weights": ["uniform", "distance"],
        "neigh_reg__algorithm": ["auto", "ball_tree", "kd_tree"],
        "neigh_reg__leaf_size": [20,30,50,100,150],
        "neigh_reg__p": [2,5,10],
        "neigh_reg__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'neigh_reg__weights': 'distance', 'neigh_reg__p': 5, 'neigh_reg__n_neighbors': 65, 'neigh_reg__metric': 'manhattan', 'neigh_reg__leaf_size': 100, 'neigh_reg__algorithm': 'auto'}
Pipeline(steps=[('neigh_reg',
                 KNeighborsRegressor(leaf_size=100, metric='manhattan',
                                     n_neighbors=65, p=5,
                                     weigh

Best Model:
```
KNeighborsRegressor(leaf_size=100, metric='manhattan',
                    n_neighbors=65, p=5,
                    weights='distance')
```

In [48]:
model = Pipeline([
    ('neigh_reg'  ,   KNeighborsRegressor(leaf_size=100, metric='manhattan',
                                            n_neighbors=65, p=5,
                                            weights='distance'))
])
model

Pipeline(steps=[('neigh_reg',
                 KNeighborsRegressor(leaf_size=100, metric='manhattan',
                                     n_neighbors=65, p=5,
                                     weights='distance'))])

In [49]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=1.000,RMSE=-0.000
(CV) R2=0.573,RMSE=-44.426


In [50]:
eval.show_result(y_test, model.predict(X_test))

R2=0.321,RMSE=-34.255


It overfitted the training set.

## Non-linear RUL

In [51]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [52]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "trf_reg__regressor__weights": ["uniform", "distance"],
        "trf_reg__regressor__algorithm": ["auto", "ball_tree", "kd_tree"],
        "trf_reg__regressor__leaf_size": [20,30,50,100,150],
        "trf_reg__regressor__p": [2,5,10],
        "trf_reg__regressor__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 85}, 'trf_reg__regressor__weights': 'distance', 'trf_reg__regressor__p': 5, 'trf_reg__regressor__n_neighbors': 25, 'trf_reg__regressor__metric': 'chebyshev', 'trf_reg__regressor__leaf_size': 100, 'trf_reg__regressor__algorithm': 'kd_tree'}
Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 85}}

KNeighborsRegressor(algorithm='kd_tree',
                    leaf_size=100,
                    metric='chebyshev',
                    n_neighbors=25,
                    p=5,
                    weights='distance'),
```



In [53]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(algorithm='kd_tree',
                                            leaf_size=100,
                                            metric='chebyshev',
                                            n_neighbors=25,
                                            p=5,
                                            weights='distance'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':85})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KNeighborsRegressor(algorithm='kd_tree',
                                                                          leaf_size=100,
                                                                          metric='chebyshev',
                                                                          n_neighbors=25,
                                                                          p=5,
                                                                          weights='distance'),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f6feded6cb0>,
                                                                            kw_args={'a_max': 85,
                                                                                     'a_min': 0})))])

In [54]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=1.000,RMSE=-0.000
(CV) R2=0.782,RMSE=-12.280


In [55]:
eval.show_result(y_test, model.predict(X_test))

R2=0.570,RMSE=-27.259


Didn't overfit as much. Generalized better.

## PolyFeatures + Linear RUL

In [56]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('neigh_reg'  ,   KNeighborsRegressor())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('neigh_reg', KNeighborsRegressor())])

In [57]:
# ~12min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "neigh_reg__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "neigh_reg__weights": ["uniform", "distance"],
        "neigh_reg__algorithm": ["auto", "ball_tree", "kd_tree"],
        "neigh_reg__leaf_size": [20,30,50,100,150],
        "neigh_reg__p": [2,5,10],
        "neigh_reg__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 2, 'neigh_reg__weights': 'distance', 'neigh_reg__p': 5, 'neigh_reg__n_neighbors': 75, 'neigh_reg__metric': 'manhattan', 'neigh_reg__leaf_size': 100, 'neigh_reg__algorithm': 'auto'}
Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
    

Best Model:
```
PolynomialFeatures(include_bias=False, interaction_only=True))
KNeighborsRegressor(leaf_size=100, metric='manhattan',
                    n_neighbors=75, p=5,
                    weights='distance'))
```


In [58]:
model = Pipeline([
    ('poly_ft' , PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('neigh_reg' ,   KNeighborsRegressor(leaf_size=100, metric='manhattan',
                                     n_neighbors=75, p=5,
                                     weights='distance'))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('neigh_reg',
                 KNeighborsRegressor(leaf_size=100, metric='manhattan',
                                     n_neighbors=75, p=5,
                                     weights='distance'))])

In [59]:
# ~5min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=1.000,RMSE=-0.000
(CV) R2=0.577,RMSE=-44.237


In [61]:
eval.show_result(y_test, model.predict(X_test))

R2=0.322,RMSE=-34.215


No clear benefit from polynomial features in linear RUL, as with the other models. 

## PolyFeatures + Non-Linear RUL

In [62]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [63]:
# ~8min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "trf_reg__regressor__weights": ["uniform", "distance"],
        "trf_reg__regressor__algorithm": ["auto", "ball_tree", "kd_tree"],
        "trf_reg__regressor__leaf_size": [20,30,50,100,150],
        "trf_reg__regressor__p": [2,5,10],
        "trf_reg__regressor__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 81}, 'trf_reg__regressor__weights': 'uniform', 'trf_reg__regressor__p': 10, 'trf_reg__regressor__n_neighbors': 55, 'trf_reg__regressor__metric': 'manhattan', 'trf_reg__regressor__leaf_size': 20, 'trf_reg__regressor__algorithm': 'auto', 'poly_ft__interaction_only': True, 'poly_ft__include_bias': True, 'poly_ft__degree': 2}
P

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 81}
PolynomialFeatures(interaction_only=True)
KNeighborsRegressor(leaf_size=20,
                    metric='manhattan',
                    n_neighbors=55,
                    p=10),
```


In [66]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(leaf_size=20,
                                            metric='manhattan',
                                            n_neighbors=55,
                                            p=10),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':81})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KNeighborsRegressor(leaf_size=20,
                                                                          metric='manhattan',
                                                                          n_neighbors=55,
                                                                          p=10),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f6feded6cb0>,
                                                                            kw_args={'a_max': 81,
                                                                                     'a_min': 0})))])

In [67]:
# ~5min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.838,RMSE=-9.978
(CV) R2=0.822,RMSE=-10.409


In [68]:
eval.show_result(y_test, model.predict(X_test))

R2=0.530,RMSE=-28.481


Using the polynominal features actually worsened the perfomance a little bit, when comparing with not using them and the non-linear RUL.